In [4]:
# Dependencies
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import linregress
import numpy as np
import pandas as pd
from prophet import Prophet
import datetime as dt
from IPython.display import display
%matplotlib inline

In [22]:
df = pd.read_csv("Resources/HR_Analytics.csv").dropna()

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1423 entries, 0 to 1479
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmpID                     1423 non-null   object 
 1   Age                       1423 non-null   int64  
 2   AgeGroup                  1423 non-null   object 
 3   Attrition                 1423 non-null   object 
 4   BusinessTravel            1423 non-null   object 
 5   DailyRate                 1423 non-null   int64  
 6   Department                1423 non-null   object 
 7   DistanceFromHome          1423 non-null   int64  
 8   Education                 1423 non-null   int64  
 9   EducationField            1423 non-null   object 
 10  EmployeeCount             1423 non-null   int64  
 11  EmployeeNumber            1423 non-null   int64  
 12  EnvironmentSatisfaction   1423 non-null   int64  
 13  Gender                    1423 non-null   object 
 14  HourlyRate   

In [24]:
df.columns

Index(['EmpID', 'Age', 'AgeGroup', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'SalarySlab',
       'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StandardHours', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [25]:
df.head()

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,...,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,...,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,...,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,...,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,...,4,80,0,0,0,3,0,0,0,0.0


In [53]:
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1423.000000,1423.000000,1423.000000,1423.000000,1423.0,1423.00000,1423.000000,1423.000000,1423.000000,1423.000000,...,1423.000000,1423.0,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000
mean,36.924807,802.000000,9.262825,2.907238,1.0,1063.66409,2.726634,66.000703,2.726634,2.070274,...,2.701335,80.0,0.797611,11.302881,2.799719,2.761771,7.042164,4.255095,2.211525,4.118060
std,9.133367,404.008071,8.146760,1.023547,0.0,595.37789,1.089736,20.373292,0.712882,1.107122,...,1.081122,0.0,0.850958,7.819449,1.288246,0.710698,6.147705,3.631051,3.248210,3.555484
min,18.000000,102.000000,1.000000,1.000000,1.0,1.00000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,550.50000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1066.00000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1587.50000,4.000000,84.000000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.00000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [52]:
for col in df.columns:
    if len(df[col].unique()) <= 10:
        print(f"{col}: {df[col].unique()}")

AgeGroup: ['18-25' '26-35' '36-45' '46-55' '55+']
Attrition: ['Yes' 'No']
BusinessTravel: ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel' 'TravelRarely']
Department: ['Research & Development' 'Sales' 'Human Resources']
Education: [3 2 1 4 5]
EducationField: ['Life Sciences' 'Medical' 'Marketing' 'Technical Degree' 'Other'
 'Human Resources']
EmployeeCount: [1]
EnvironmentSatisfaction: [3 4 2 1]
Gender: ['Male' 'Female']
JobInvolvement: [3 2 1 4]
JobLevel: [1 2 3 4 5]
JobRole: ['Laboratory Technician' 'Sales Representative' 'Research Scientist'
 'Human Resources' 'Manufacturing Director' 'Sales Executive'
 'Healthcare Representative' 'Research Director' 'Manager']
JobSatisfaction: [3 2 4 1]
MaritalStatus: ['Single' 'Divorced' 'Married']
SalarySlab: ['Upto 5k' '5k-10k' '10k-15k' '15k+']
NumCompaniesWorked: [1 0 5 6 3 2 9 8 7 4]
Over18: ['Y']
OverTime: ['No' 'Yes']
PerformanceRating: [3 4]
RelationshipSatisfaction: [3 1 4 2]
StandardHours: [80]
StockOptionLevel: [0 2 1 3]
TrainingTimesL

In [65]:
df.groupby(df['AgeGroup']).describe()

Age                                                    DailyRate  \
          count       mean       std   min   25%   50%   75%   max     count   
AgeGroup                                                                       
18-25     117.0  22.367521  2.222915  18.0  21.0  23.0  24.0  25.0     117.0   
26-35     590.0  30.972881  2.831297  26.0  29.0  31.0  34.0  35.0     590.0   
36-45     449.0  39.944321  2.863567  36.0  37.0  40.0  42.0  45.0     449.0   
46-55     223.0  50.130045  2.921826  46.0  48.0  50.0  53.0  55.0     223.0   
55+        44.0  57.704545  1.390659  56.0  56.0  58.0  59.0  60.0      44.0   

                      ... YearsSinceLastPromotion       YearsWithCurrManager  \
                mean  ...                     75%   max                count   
AgeGroup              ...                                                      
18-25     772.025641  ...                     1.0   5.0                117.0   
26-35     797.579661  ...                     2.0  11.0                590.0   
36-45     805.619154  ...                     4.0  15.0                449.0   
46-55     824.242152  ...                     4.0  15.0                223.0   
55+       791.318182  ...                     5.0  15.0                 44.0   

                                                         
              mean       std  min   25%  50%  75%   max  
AgeGroup                                                 
18-25     1.487179  1.494907  0.0  0.00  2.0  2.0   7.0  
26-35     3.864407  3.092560  0.0  2.00  3.0  7.0  15.0  
36-45     4.721604  3.970932  0.0  2.00  3.0  7.0  17.0  
46-55     4.932735  3.829703  0.0  2.00  4.0  7.0  17.0  
55+       4.227273  3.747021  0.0  1.75  3.0  8.0  14.0  

[5 rows x 208 columns]

In [91]:
df["ProportionAtCompany"] = df["YearsAtCompany"] / df["TotalWorkingYears"]

In [103]:
attrition_group = df.groupby(df['Attrition'])
for attrition_status, group in attrition_group:
    print(f"Attrition: {attrition_status}")
    for col in group.columns:
        if group[col].dtype != "object":
            print(f"{col}: {group[col].mean()}")
    print("-------------------------------")

Attrition: No
Age: 37.5406538139145
DailyRate: 811.8466051969824
DistanceFromHome: 8.983235540653814
Education: 2.9186923721709976
EmployeeCount: 1.0
EmployeeNumber: 1067.799664710813
EnvironmentSatisfaction: 2.7736797988264876
HourlyRate: 66.07963118189438
JobInvolvement: 2.766135792120704
JobLevel: 2.1533948030176027
JobSatisfaction: 2.774518021793797
MonthlyIncome: 6857.795473595977
MonthlyRate: 14259.422464375524
NumCompaniesWorked: 2.658005029337804
PercentSalaryHike: 15.224643755238894
PerformanceRating: 3.151718357082984
RelationshipSatisfaction: 2.7200335289186923
StandardHours: 80.0
StockOptionLevel: 0.8516345347862532
TotalWorkingYears: 11.891869237217099
TrainingTimesLastYear: 2.8357082984073765
WorkLifeBalance: 2.7820620284995807
YearsAtCompany: 7.4174350377200335
YearsInCurrentRole: 4.5155071248952225
YearsSinceLastPromotion: 2.264040234702431
YearsWithCurrManager: 4.373009220452641
ProportionAtCompany: 0.686190492969737
-------------------------------
Attrition: Yes
Age: 